# Task 1

### Data processing

In [2]:
pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [183]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import warnings
warnings.filterwarnings('ignore')

In [205]:
#Download raw data
data = pd.read_csv('data/Groceries data train.csv')

In [206]:
data

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,3021,30/01/2015,frankfurter,2015,1,30,4
1,1292,24/10/2015,pork,2015,10,24,5
2,4206,4/04/2014,root vegetables,2014,4,4,4
3,4369,25/08/2015,onions,2015,8,25,1
4,1522,1/07/2014,waffles,2014,7,1,1
...,...,...,...,...,...,...,...
26995,4206,14/01/2014,rolls/buns,2014,1,14,1
26996,1660,15/06/2014,domestic eggs,2014,6,15,6
26997,1595,10/05/2015,liver loaf,2015,5,10,6
26998,3231,31/03/2015,sausage,2015,3,31,1


In [207]:
data.columns

Index(['Member_number', 'Date', 'itemDescription', 'year', 'month', 'day',
       'day_of_week'],
      dtype='object')

In [208]:
# Group the data by 'Member_number' and 'Date', then aggregate the 'itemDescription' into lists
transactions_data = data.groupby(['Member_number', 'Date'])['itemDescription'].apply(list).reset_index()

# Extract the list of transactions
transactions = transactions_data['itemDescription'].tolist()

# Display the first transaction as an example
transactions[0]

['sausage', 'yogurt']

### Mine frequent patterns by using Apriori algorithm

In [51]:
pip install apyori

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [211]:
from apyori import apriori
from mlxtend.frequent_patterns import association_rules
rules = apriori(transactions, min_support=0.0005, min_confidence=0.002)
Results = list(rules)
Results

[RelationRecord(items=frozenset({'Instant food products'}), support=0.003021365369397885, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Instant food products'}), confidence=0.003021365369397885, lift=1.0)]),
 RelationRecord(items=frozenset({'UHT-milk'}), support=0.015034889576289475, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'UHT-milk'}), confidence=0.015034889576289475, lift=1.0)]),
 RelationRecord(items=frozenset({'baking powder'}), support=0.006186605280195669, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'baking powder'}), confidence=0.006186605280195669, lift=1.0)]),
 RelationRecord(items=frozenset({'beef'}), support=0.02640097834688152, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'beef'}), confidence=0.02640097834688152, lift=1.0)]),
 RelationRecord(items=frozenset({'berries'}), support=0.01661750953168837, ordered_statistics=[Ord

In [212]:
df_results = pd.DataFrame(Results)
df_results

,items,support,ordered_statistics
0,(Instant food products),0.003021,"[((), (Instant food products), 0.0030213653693..."
1,(UHT-milk),0.015035,"[((), (UHT-milk), 0.015034889576289475, 1.0)]"
2,(baking powder),0.006187,"[((), (baking powder), 0.006186605280195669, 1..."
3,(beef),0.026401,"[((), (beef), 0.02640097834688152, 1.0)]"
4,(berries),0.016618,"[((), (berries), 0.01661750953168837, 1.0)]"
...,...,...,...
809,"(rolls/buns, whole milk, other vegetables)",0.000504,"[((other vegetables), (rolls/buns, whole milk)..."
810,"(rolls/buns, whole milk, sausage)",0.000719,"[((rolls/buns), (sausage, whole milk), 0.00861..."
811,"(rolls/buns, whole milk, yogurt)",0.000504,"[((rolls/buns), (whole milk, yogurt), 0.006029..."
812,"(whole milk, sausage, soda)",0.000504,"[((sausage), (soda, whole milk), 0.01112877583..."


In [213]:
# keep support in a separate data frame so we can use later
support = df_results.support

In [225]:
#all four empty list which will contain lhs, rhs, confidance and lift respectively.
first_values = []
second_values = []
third_values = []
fourth_value = []

# loop number of rows time and append 1 by 1 value in a separate list
# first and second element was frozenset which need to be converted in list
for i in range(df_results.shape[0]):
    single_list = df_results['ordered_statistics'][i][0]
    first_values.append(list(single_list[0]))
    second_values.append(list(single_list[1]))
    third_values.append(single_list[2])
    fourth_value.append(single_list[3])

In [226]:
# convert all four list into dataframe for further operation
lhs = pd.DataFrame(first_values)
rhs = pd.DataFrame(second_values)

confidance=pd.DataFrame(third_values,columns=['Confidance'])

lift=pd.DataFrame(fourth_value,columns=['lift'])

In [252]:
# concat all list together in a single dataframe
df_final = pd.concat([lhs,rhs,support,confidance,lift], axis=1)
df_final
df_final.fillna(value=' ', inplace=True)
df_final.head()

,0,0,1,support,Confidance,lift
0,,Instant food products,,0.003021,0.003021,1.0
1,,UHT-milk,,0.015035,0.015035,1.0
2,,baking powder,,0.006187,0.006187,1.0
3,,beef,,0.026401,0.026401,1.0
4,,berries,,0.016618,0.016618,1.0


In [253]:
#set column name
df_final.columns = ['lhs','rhs',1,'support','confidance','lift']
df_final.head()

,lhs,rhs,1,support,confidance,lift
0,,Instant food products,,0.003021,0.003021,1.0
1,,UHT-milk,,0.015035,0.015035,1.0
2,,baking powder,,0.006187,0.006187,1.0
3,,beef,,0.026401,0.026401,1.0
4,,berries,,0.016618,0.016618,1.0


In [254]:
# add all three column to lhs itemset only
df_final['lhs'] = df_final['lhs'] + str(", ") + df_final[1]

df_final['rhs'] = df_final['rhs']+str(", ")+ str(", ")

In [258]:
df_final.drop(columns=[1],inplace=True)

In [259]:
df_final.sort_values('lift', ascending=False).head(10)

,lhs,rhs,support,confidance,lift
144,"beef,","pasta, ,",0.000504,0.019074,3.314271
336,"chewing gum,","coffee, ,",0.000575,0.066116,2.581670
290,"cake bar,","sausage, ,",0.000504,0.116667,2.578352
428,"cream cheese ,","waffles, ,",0.000575,0.031496,2.292287
586,"long life bakery product,","napkins, ,",0.000504,0.039548,2.216762
238,"brandy,","whole milk, ,",0.000504,0.259259,2.196199
585,"liquor,","tropical fruit, ,",0.000575,0.105263,2.174239
331,"cat food,","fruit/vegetable juice, ,",0.000504,0.056452,2.173778
497,"frankfurter,","red/blush wine, ,",0.000504,0.017456,2.110094
116,"UHT-milk,","cream cheese , ,",0.000575,0.038278,2.094865


In [260]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [261]:
df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13896,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13897,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13898,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13899,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [262]:
frequent_itemsets = fpgrowth(df, min_support=0.0005, use_colnames=True)
frequent_itemsets

from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(yogurt),(whole milk),0.064312,0.118049,0.006546,0.101790,0.862266,-0.001046,0.981898,-0.145820
1,"(rolls/buns, yogurt)",(whole milk),0.004460,0.118049,0.000504,0.112903,0.956409,-0.000023,0.994199,-0.043777
2,(sausage),(whole milk),0.045249,0.118049,0.005251,0.116057,0.983127,-0.000090,0.997747,-0.017658
3,"(sausage, yogurt)",(whole milk),0.003237,0.118049,0.000504,0.155556,1.317720,0.000121,1.044416,0.241896
4,"(sausage, rolls/buns)",(whole milk),0.003021,0.118049,0.000719,0.238095,2.016918,0.000363,1.157561,0.505722
5,"(sausage, whole milk)",(rolls/buns),0.005251,0.083519,0.000719,0.136986,1.640178,0.000281,1.061954,0.392371
6,"(sausage, soda)",(whole milk),0.003309,0.118049,0.000504,0.152174,1.289073,0.000113,1.040250,0.224994
7,(white bread),(whole milk),0.017769,0.118049,0.001870,0.105263,0.891690,-0.000227,0.985710,-0.110054
8,"(rolls/buns, other vegetables)",(whole milk),0.005036,0.118049,0.000504,0.100000,0.847105,-0.000091,0.979945,-0.153550
9,(frankfurter),(whole milk),0.028847,0.118049,0.003237,0.112219,0.950617,-0.000168,0.993434,-0.050775


In [263]:
def predict_items(purchased_items):
    # create a list to store the predicted items
    predicted_items = []
    
    # iterate over the top rules
    for index, row in rules.iterrows():
        # get the items in the antecedent and consequent of the rule
        antecedent = row["antecedents"]
        consequent = row["consequents"]
        
        # check if all the items in the antecedent are in the purchased items
        if antecedent.issubset(purchased_items):
            # add the items in the consequent to the predicted items
            predicted_items.append(consequent)
    
    # return the list of predicted items
    return predicted_items

In [264]:
# make a prediction for a sample set of purchased items
purchased_items = {"yogurt", "whole milk"}
predicted_items = predict_items(purchased_items)

# print the predicted items to the console
print(predicted_items)

[frozenset({'whole milk'})]


In [265]:
# make a prediction for a sample set of purchased items
purchased_items = {"yogurt"}
predicted_items = predict_items(purchased_items)

# print the predicted items to the console
print(predicted_items)

[frozenset({'whole milk'})]
